# Rozpoznawaine znaków drogowych - przykład zabawy Data Science
#  [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/)

## Najpierw klasycznie załączamy biblioteki

In [ ]:
import pickle

import numpy as np
np.random.seed(108)
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

import matplotlib.pyplot as plt

%matplotlib inline

## Wczytujemy dane

In [ ]:
training_file = "input/train.p"
test_file = "input/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)

with open(test_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

print(X_train.shape, X_test.shape)

In [ ]:
signnames = pd.read_csv('input/signnames.csv')
signnames.head(50)

In [ ]:
traffic_labels_dict = signnames[['b']].to_dict()['b']

In [ ]:
for sign_id in traffic_labels_dict.keys():
    print(f'===== {traffic_labels_dict[sign_id]} =====')
    signs = X_train[y_train == sign_id]
    
    plt.figure(figsize=(15, 12))
    plt.subplot('191')
    plt.imshow(signs[1,])
    plt.axis('off')

    plt.tight_layout()
    plt.show()

## Przetwarzamy dane, tak by można odpalić na nich nasz model

In [ ]:
if y_train.ndim == 1: 
    y_train = to_categorical(y_train)
    
if y_test.ndim == 1: 
    y_test = to_categorical(y_test)    
    
input_shape = X_train.shape[1:]
num_classes = len(traffic_labels_dict)

## Tworzymy architekturę modelu (układamy klocki lego)

In [ ]:
def doubled_cnn3_with_bn():
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
        
    model.add(Flatten())
        
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model


model = doubled_cnn3_with_bn()
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Do tworzenia wizualizacji uczenia
tensorboard = TensorBoard('./logs/live', write_images=False)

## Trening, uczymy naszą sieć

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=3,
                    verbose=1,
                    validation_data=(X_test, y_test),
                    callbacks=[tensorboard]
                   )

## Wizualizacja uczenia

In [ ]:
!tensorboard --logdir logs/live

In [ ]:
def draw_learning_curve(history, key='acc', ylim=(0.8, 1.01)):
    plt.figure(figsize=(12,6))
    plt.plot(history.history[key])
    plt.plot(history.history['val_' + key])
    plt.title('Learning Curve')
    plt.ylabel(key.title())
    plt.xlabel('Epoch')
    plt.ylim(ylim)
    plt.legend(['train', 'test'], loc='best')
    plt.show()
    
# draw_learning_curve(history, key='acc', ylim=(0.5, 1.))
# draw_learning_curve(history, key='loss', ylim=(0., 1.))

## Testujemy, sprawdzamy czego tak naprawdę się nauczyła


### Średni wynik ludzi:
#### Accuracy: 98.84%
#### Error: 1,16%

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

# print(f'Test loss: {round(score[0], 3)}')

print(f'Test accuracy: {round(score[1], 4)*100}%')

print(f'Error: {round(100-score[1]*100, 2)}%')

## A teraz sprawdżmy to na przykładach

In [ ]:
for _ in range(5):
    index = np.random.choice(X_train.shape[0])
    picture = X_train[index]
    actual_sign_id = np.argmax(y_train[index])
    predicted_sign_id = np.argmax(model.predict(np.reshape(X_train[index], (1, 32, 32, 3))))
    
    print('='*30)
    
    plt.figure(figsize=(15, 12))
    plt.subplot('191')
    plt.imshow(picture)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f'Actual: {traffic_labels_dict[actual_sign_id]}')
    print(f'Predicted: {traffic_labels_dict[predicted_sign_id]}')